# Some code that allowed to make conclusions made in README file

It is here to allow people to verify that the claims made in the README file are not baseless.
The code quality here barely allows reading and understanding what is going on, and it is not suggested to do so.

In [13]:


import os
import csv
import numpy as np
import pandas as pd
from glob import glob
from collections import OrderedDict
import matplotlib.pyplot as plt

data_dir = "data"

all_dfs = OrderedDict()

# read data
class LineWrapper:
    def __init__(self, 
                 detector, 
                 descriptor, 
                 descriptor_type,
                 matcher,
                 selector,
                 lst):
        self._lst = lst
        self._detector = detector
        self._descriptor = descriptor
        self._descriptor_type = descriptor_type
        self._matcher = matcher
        self._selector = selector
        
    def detector(self):
        return self._detector
    
    def descriptor(self):
        return self._descriptor
    
    def descriptor_type(self):
        return self._descriptor_type
    
    def matcher(self):
        return self._matcher
    
    def selector(self):
        return self._selector
        
    def prev_img(self):
        return int(self._lst[0]) - 1
    
    def cur_img(self):
        return int(self._lst[0])
    
    def lidar_ttc(self):
        return float(self._lst[1])
    
    def camera_ttc(self):
        return float(self._lst[2])

files = glob(os.path.join(data_dir, '*.txt'))
for file in files:
    file_name = os.path.basename(file).split('.')[0]
    det, desc, desc_type, mat, sel = file_name.split('_')
    
    data = pd.read_csv(file, sep=' ', header=0)

    if   data.isna().values.sum() \
       + data.isin([-np.inf, np.inf]).values.sum() \
       + data.isnull().values.sum() == 0 \
       and \
       ( data['ttc_camera'] < 5).values.sum() == 0  \
       and  \
       ( data['ttc_camera'] > 25).values.sum() == 0  \
       and \
       ( data['ttc_lidar'] < 5).values.sum() == 0  \
       and  \
       ( data['ttc_lidar'] > 25).values.sum() == 0:
        
        data['ttc_avg'] = (data['ttc_lidar'] + data['ttc_camera']) / 2.0
        data['ttc_diff'] = data['ttc_lidar'] - data['ttc_camera']
        
        data['delta_ttc_avg']   = np.hstack((0, data['ttc_avg'].values[1:]   - data['ttc_avg'].values[:-1]))
        data['delta_ttc_lidar'] = np.hstack((0, data['ttc_lidar'].values[1:] - data['ttc_lidar'].values[:-1]))
        data['delta_ttc_camera'] = np.hstack((0, data['ttc_camera'].values[1:] - data['ttc_camera'].values[:-1]))
    
    
        all_dfs[file_name] = {
            'data': data, 
            'count': len(data),
            'mae_ttc_diff':          data['ttc_diff'].abs().mean(),
            'rmse_delta_ttc_lidar':  ((data['delta_ttc_lidar'][1:]) ** 2).mean() ** 0.5,
            'rmse_delta_ttc_camera': ((data['delta_ttc_camera'][1:]) ** 2).mean() ** 0.5,
            'rmse_delta_ttc_avg':    ((data['delta_ttc_avg'][1:]) ** 2).mean() ** 0.5,
        }
        all_dfs[file_name]['sum_of_rmses'] =                \
            all_dfs[file_name][ 'rmse_delta_ttc_lidar'  ] + \
            all_dfs[file_name][ 'rmse_delta_ttc_camera' ] + \
            all_dfs[file_name][ 'rmse_delta_ttc_avg'    ]
    else:
        pass #print(file_name, '\n', data, '\n')

min_mae,  max_mae  = None, None
min_rmse, max_rmse = None, None
min_count, max_count = None, None
for key, val in all_dfs.items():
    if not min_mae or min_mae > val['mae_ttc_diff']:
        min_mae = val['mae_ttc_diff']
    if not max_mae or max_mae < val['mae_ttc_diff']:
        max_mae = val['mae_ttc_diff']

    if not min_rmse or min_rmse > val['sum_of_rmses']:
        min_rmse = val['sum_of_rmses']
    if not max_rmse or max_rmse < val['sum_of_rmses']:
        max_rmse = val['sum_of_rmses']

    if not min_count or min_count > val['count']:
        min_count = val['count']
    if not max_count or max_count < val['count']:
        max_count = val['count']


assert None not in [min_mae, max_mae, min_rmse, max_rmse, min_count, max_count]
assert min_mae <= max_mae
assert min_rmse <= max_rmse
assert min_count <= max_count

mae_score   = lambda mae:   1 - (mae   - min_mae)   / (max_mae   - min_mae)
rmse_score  = lambda rmse:  1 - (rmse  - min_rmse)  / (max_rmse  - min_rmse)
count_score = lambda count: (count - min_count) / (max_count - min_count)

for combination_id, data in all_dfs.items():
    data['mae_score']   = mae_score(   data['mae_ttc_diff'] )
    data['rmse_score']  = rmse_score(  data['sum_of_rmses'] )
    data['count_score'] = count_score( data['count']        )
    data['total_score'] = 1./3. * (data['mae_score'] + data['rmse_score'] + data['count_score'])

all_dfs = OrderedDict(reversed(sorted(list(all_dfs.items()), key=lambda key_val: key_val[1]['total_score'])))


for key, val in all_dfs.items():
    print(f"combination_id = {key}\n",
          f"total_score = {val['total_score']}\n",
          f"mae_score   = {val['mae_score']}\n",
          f"rmse_score  = {val['rmse_score']}\n",
          f"count_score = {val['count_score']}\n",
          val['data'], '\n')

    
    
            
# fig, ax = plt.subplots(1, 1, figsize=(20, 20))
# label_helper = dict()
# for combination_id, combination_dict in data.items():
#     x = np.arange(0, len(combination_dict), 1)
#     y = list(combination_dict.values())
    
#     if not label_helper.get(tuple(y), None):
#         label_helper[tuple(y)] = []
#     label_helper[tuple(y)].append(combination_id[1:].split(')')[0])

# for combination_id, combination_dict in data.items():
#     x = np.arange(0, len(combination_dict), 1)
#     y = list(combination_dict.values())
    
#     ax.plot(x, y,
#             label=combination_id + ": min=" + str(min(combination_dict.values()))
#                   + " mean=" + str(sum(combination_dict.values()) // len(combination_dict.values())) 
#                   + " max=" + str(max(combination_dict.values())))
    
#     ax.set_xticks(np.arange(0, len(combination_dict), 1))
#     ax.set_xticklabels(combination_dict.keys())
#     ax.set_xlabel('image pairs')
#     ax.set_ylabel('number of matches')
    
# pos = 0
# for line, ids in label_helper.items():
#     ax.annotate('(' + ','.join(ids) + ')', # this is the text
#                 (pos, line[pos]), # this is the point to label
#                  textcoords="offset points", # how to position the text
#                  xytext=(0,0), # distance from text to points (x,y)
#                  ha='center') # horizontal alignment can be left, right or center
#     pos = (pos + 1) % 9
    
# ax.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
#            ncol=1, mode="expand", borderaxespad=0.)

# plt.show()

combination_id = SHITOMASI_SIFT_HOG_FLANN_NN
 total_score = 0.9761945633341825
 mae_score   = 0.9488615927725769
 rmse_score  = 0.9797220972299707
 count_score = 1.0
     image_id  ttc_lidar  ttc_camera    ttc_avg  ttc_diff  delta_ttc_avg  \
0          1   14.87490     14.1124  14.493650   0.76250       0.000000   
1          2   12.23420     13.0387  12.636450  -0.80450      -1.857200   
2          4   13.00870     12.8675  12.938100   0.14120       0.301650   
3          5   11.86800     11.9557  11.911850  -0.08770      -1.026250   
4          6   10.61420     13.5822  12.098200  -2.96800       0.186350   
5          7   15.24570     13.2947  14.270200   1.95100       2.172000   
6          8   15.65190     13.9921  14.822000   1.65980       0.551800   
7          9   12.34920     12.0542  12.201700   0.29500      -2.620300   
8         10   14.68970     13.6672  14.178450   1.02250       1.976750   
9         11   12.71550     12.0012  12.358350   0.71430      -1.820100   
10      

 count_score = 0.42857142857142855
     image_id  ttc_lidar  ttc_camera    ttc_avg  ttc_diff  delta_ttc_avg  \
0          2   12.23420    17.20320  14.718700  -4.96900       0.000000   
1          4   13.00870    16.98920  14.998950  -3.98050       0.280250   
2          6   10.61420    17.67430  14.144250  -7.06010      -0.854700   
3          7   15.24570    15.84560  15.545650  -0.59990       1.401400   
4          8   15.65190    17.89800  16.774950  -2.24610       1.229300   
5          9   12.34920    14.65050  13.499850  -2.30130      -3.275100   
6         10   14.68970    16.05990  15.374800  -1.37020       1.874950   
7         11   12.71550    12.02310  12.369300   0.69240      -3.005500   
8         13    8.89039    12.62390  10.757145  -3.73351      -1.612155   
9         14    9.95430    12.30250  11.128400  -2.34820       0.371255   
10        16   10.36950    10.80840  10.588950  -0.43890      -0.539450   
11        17   10.88900     9.04955   9.969275   1.83945      -0